In [42]:
from scipy import optimize,arange
from numpy import array
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('seaborn-whitegrid')
!pip install tabulate
from tabulate import tabulate
import sympy as sm

## Baseline model - Cournot Competition

In [43]:
def profit(xi,xj):
    """
    This profit function is of the original form and is used to find the optimal production.
    
    xi: The good produced by firm i
    xj: The good produced by firm j
    """
    return demand_good_x(xi,xj)*xi-cost_good_x(xi)

In [51]:
i = sm.symbols('i')
j = sm.symbols('j')

In [52]:
profit_i = (120-(i+j))*i-30*i
profit_j = (120-(i+j))*j-30*j

In [53]:
foci = sm.diff(profit_i,i)
focj = sm.diff(profit_j,j)

In [54]:
foci_eq = sm.Eq(foci,0)
focj_eq = sm.Eq(focj,0)
display(foci_eq, focj_eq)

Eq(-2*i - j + 90, 0)

Eq(-i - 2*j + 90, 0)

In [55]:
insert = foci_eq.subs(j,brj)
display(insert)

Eq(45 - 3*i/2, 0)

In [56]:
x = sm.solve(insert,i)
display(x)

[30]

In [57]:
def demand_good_x(xi,xj):
    """
    This function decides the demand given the quantity produced by firms i and j. This means that this function decides
    the price of the good. This works because the goods are homogenous and operate under the same market.
    
    xi: The good produced by firm i
    xj: The good produced by firm j
    120: This constant defines the quantity for which demand and thereby the price is equal to zero
    """
    return 120-(xi+xj)


def cost_good_x(x):
    """
    This is the cost function that defines the production costs of good x given the amount produced. 
    The function is of such form that is only catches variable cost, and that there therefore is no fixed costs.
    
    x: the quantity of good x
    """
    return 30*x

In [ ]:
def profit(xi,xj):
    """
    This profit function is of the original form and is used to find the optimal production.
    
    xi: The good produced by firm i
    xj: The good produced by firm j
    """
    return demand_good_x(xi,xj)*xi-cost_good_x(xi)

In [ ]:
def best_response(xj):
    """
    This function is derived from the profit function by taking the derevation with regards to xi and isolating xi.
    
    xi: The good produced by firm i
    xj: The good produced by firm j
    """
    xi = (90-xj)/2
    return xi

In [ ]:
def vector_best_response(x):
    """
    Now we take the best response functions and generating a vector containing them this is then used to solve the system
    of functions that are driven from earlier functions. Because we are looking for the vector best responce, we minus the
    response functions from an x.
    
    x: creating the difference equation that is optimized.
    """
    return array(x)-array([best_response(x[1]),best_response(x[0])])

In [ ]:
x0 = [40, 40]
results = optimize.fsolve(vector_best_response, x0)
print(results)

In [ ]:
print('The profit is', profit(results[0],results[1]))

In [ ]:
data = [["Nash Equilibrium", 30, 30 , 900]]
col_names = [" ", "Firm i's optimal goods", "Firm j's optimal goods", "Profit in Nash"]
print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))

## Extension 1 - Collusion

In [58]:
profit_i = (120-(i+j))*i-30*i

In [59]:
profit_x = profit_i.subs(j,i)

In [62]:
maximize = sm.diff(profit_x,i)

In [66]:
x_value = sm.solve(maximize,i)[0]
display(x_value)

In [69]:
profit_total = profit_x.subs(i,x_value)
display('Profit is', profit_total)

'Profit is'

2025/2

In [ ]:
def negative_profit_collusion(x): #Negative, because the minimizer underneath needs the negative value to find maximum.
    return -(90*x-2*x**2)

In [ ]:
res = optimize.minimize_scalar(negative_profit_collusion)
optimum_quantity = res.x
optimum_profit = profit(optimum_quantity,optimum_quantity)
print(optimum_quantity, optimum_profit)

In [ ]:
print('The optimal proft occurs when the two companies collude and produce', "%.2f" % optimum_quantity, 'units and gaining', "%.2f" % optimum_profit, 'as profits')

## Extension 2 - Deviation from collusion

In [ ]:
x_grid_deviate = np.linspace(10, 60, 100)
xvalue_deviate = []
profits_deviate = []

for x in x_grid_deviate:
    xvalue_deviate.append(x)
    profits_deviate.append(profit(x,optimum_quantity))

In [ ]:
optimum_deviated_profit = max(profits_deviate)
optimum_deviated_quantity = xvalue_deviate[profits_deviate.index(max(profits_deviate))]

fig = plt.figure(figsize=(8, 5), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(xvalue_deviate,profits_deviate)
ax.set_xlabel('x-value', fontsize=12)
ax.set_ylabel('Profit', fontsize=12)
ax.set_title('Profit under deviation from collusion', fontsize=18, color='black')
ax.axvline(x=optimum_deviated_quantity, color='red',linestyle = 'dashed')
ax.scatter(optimum_deviated_quantity,optimum_deviated_profit, color='black')
ax.fill_between(xvalue_deviate,profits_deviate, color='pink', alpha=0.15)

In [ ]:
print('Profit =', optimum_deviated_profit, 'and quantity for firm i = ', optimum_deviated_quantity)

In [ ]:
profit_collude = optimum_profit+optimum_profit
profit_deviate = optimum_deviated_profit+profit(optimum_quantity,optimum_deviated_quantity)

In [ ]:
print(profit_collude, profit_deviate)

## Simulation - Trigger and Tit for tat strategy

### Trigger strategy

The trigger strategy is a strategy that is used in the cournot problem. Basically, the two firms decide together that they are going til use the collusion strategy where they will get a bigger profit than if they just did the Nash equilibrium. This is conditioned on that fact that there is incentive to deviate. This means that if firm i decides to deviate, he will get a larger profit in one period, but then firm j will punish this behaviour by playing the Nash equilibrium where there is no incentive to deviate. This can be shown as seen below.

In [ ]:
t_grid = np.linspace(0, 49, 50)

#Triggger
profit_i_trig = []
profit_j_trig = []

for i in t_grid:
    if i <= 20:
        profit_i_trig.append(optimum_profit)
        profit_j_trig.append(optimum_profit)
    elif i == 21:
        profit_i_trig.append(optimum_deviated_profit)
        profit_j_trig.append(profit(optimum_quantity,optimum_deviated_quantity))
    else:
        profit_i_trig.append(profit(results[0],results[1]))
        profit_j_trig.append(profit(results[0],results[1]))

In [ ]:
fig = plt.figure(figsize=(8, 5), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(t_grid,profit_i_trig, profit_j_trig)

In [ ]:
trig_i_cumulative = np.cumsum(profit_i_trig)
trig_j_cumulative = np.cumsum(profit_j_trig)

In [ ]:
fig = plt.figure(figsize=(12, 8), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(t_grid,trig_i_cumulative, trig_j_cumulative)

In [ ]:
t_grid = np.linspace(0, 49, 50)
d_grid = np.linspace(0, 49, 50)

#Triggger
profit_i_trig_total = []
profit_j_trig_total = []

for j in d_grid:
    
    profit_i_trig = []
    profit_j_trig = []

    for i in t_grid:
        if i <= (j-1):
            profit_i_trig.append(optimum_profit)
            profit_j_trig.append(optimum_profit)
        elif i == j:
            profit_i_trig.append(optimum_deviated_profit)
            profit_j_trig.append(profit(optimum_quantity,optimum_deviated_quantity))
        else:
            profit_i_trig.append(profit(results[0],results[1]))
            profit_j_trig.append(profit(results[0],results[1]))
            
    profit_i_trig_total.append(sum(profit_i_trig))
    profit_j_trig_total.append(sum(profit_j_trig))

In [ ]:
optimum_profit_trig = max(profit_i_trig_total)
optimum_deviationperiod_trig = d_grid[profit_i_trig_total.index(max(profit_i_trig_total))]

fig = plt.figure(figsize=(12, 8), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(d_grid,profit_i_trig_total, lw=2,color='blue',label='Profit for firm i (The deviator)')
ax.plot(d_grid,profit_j_trig_total, lw=2, color='red',label='Profit for firm j')
ax.legend(loc='lower left',frameon=True)
ax.plot(d_grid,profit_i_trig_total, profit_j_trig_total)
ax.axvline(x=optimum_deviationperiod_trig, color='red',linestyle = 'dashed')
ax.scatter(optimum_deviationperiod_trig,optimum_profit_trig, color='black')
ax.fill_between(d_grid,profit_i_trig_total, color='yellow', alpha=0.15)
ax.set(xlim=(0, 50), ylim=(44000, 51000))

In [ ]:
print(optimum_profit_trig, optimum_deviationperiod_trig)

### Tit For Tat strategy

The Tit For Tat strategy means that the one firm copies the other firms behaviour from the earlier period. This means in practice that they start at the Nash equilibrium and at some point one of the firms deviates from the strategy. This results in that the next period, they copy 

In [ ]:
t_grid = np.linspace(0, 49, 50)

#Triggger
profit_i_tft = []
profit_j_tft = []

for i in t_grid:
    if i <= 20:
        profit_i_tft.append(optimum_profit)
        profit_j_tft.append(optimum_profit)
    elif i == 21:
        profit_i_tft.append(optimum_deviated_profit)
        profit_j_tft.append(profit(optimum_quantity,optimum_deviated_quantity))
    elif (i%2 == 0):
        profit_i_tft.append(profit(optimum_quantity,optimum_deviated_quantity))
        profit_j_tft.append(optimum_deviated_profit)
    else:
        profit_i_tft.append(optimum_deviated_profit)
        profit_j_tft.append(profit(optimum_quantity,optimum_deviated_quantity))

In [ ]:
fig = plt.figure(figsize=(8, 5), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(t_grid,profit_i_tft, profit_j_tft)

In [ ]:
tft_i_cumulative = np.cumsum(profit_i_tft)
tft_j_cumulative = np.cumsum(profit_j_tft)

In [ ]:
fig = plt.figure(figsize=(12, 8), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(t_grid,tft_i_cumulative, tft_j_cumulative)

In [ ]:
t_grid = np.linspace(0, 49, 50)
d_grid = np.linspace(0, 49, 50)

#Triggger
profit_i_tft_total = []
profit_j_tft_total = []

for j in d_grid:
    
    profit_i_tft = []
    profit_j_tft = []
    
    for i in t_grid:
        if i <= (j-1):
            profit_i_tft.append(optimum_profit)
            profit_j_tft.append(optimum_profit)
        elif i == j:
            profit_i_tft.append(optimum_deviated_profit)
            profit_j_tft.append(profit(optimum_quantity,optimum_deviated_quantity))
        elif (i%2 == 0):
            profit_i_tft.append(profit(optimum_quantity,optimum_deviated_quantity))
            profit_j_tft.append(optimum_deviated_profit)
        else:
            profit_i_tft.append(optimum_deviated_profit)
            profit_j_tft.append(profit(optimum_quantity,optimum_deviated_quantity))
            
    profit_i_tft_total.append(sum(profit_i_tft))
    profit_j_tft_total.append(sum(profit_j_tft))

In [ ]:
optimum_profit_tft = max(profit_i_tft_total)
optimum_deviationperiod_tft = d_grid[profit_i_tft_total.index(max(profit_i_tft_total))]

fig = plt.figure(figsize=(12, 8), dpi=80)
ax = fig.add_subplot(1,1,1)
ax.plot(d_grid,profit_i_tft_total, lw=2,color='blue',label='Profit for firm i (The deviator)')
ax.plot(d_grid,profit_j_tft_total, lw=2, color='red',label='Profit for firm j')
ax.legend(loc='lower left',frameon=True)
ax.plot(d_grid,profit_i_tft_total, profit_j_tft_total)
ax.axvline(x=optimum_deviationperiod_tft, color='hotpink',linestyle = 'dashed')
ax.scatter(optimum_deviationperiod_tft,optimum_profit_tft, color='black')
ax.fill_between(d_grid,profit_i_tft_total, color='orange', alpha=0.15)
ax.set(xlim=(0, 50), ylim=(44000, 51000))

In [ ]:
print(optimum_profit_tft, optimum_deviationperiod_tft)

### Sumation

In [ ]:
data = [["Trigger", optimum_profit_trig , optimum_deviationperiod_trig], 
        ["Tit For Tat", optimum_profit_tft, optimum_deviationperiod_tft]]
col_names = ["Strategy", "Profit", "Deviation period"]
print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))